In [ ]:
genome = ''.join(line.strip() for line in open('lambda_virus.fa') if not line.startswith('>'))
print len(genome)

In [ ]:
import collections
collections.Counter(genome)

In [ ]:
def reverseComplement(seq):
    return ''.join({'A':'T','C':'G','G':'C','T':'A','N':'N'}[c] for c in seq[::-1])

In [ ]:
def isMatchAt(genome, fragment, offset, maxMismatches = 0):
    misMatches = []
    for i in range(len(fragment)):
        if genome[offset + i] != fragment[i]:
            misMatches.append(i)
            if len(misMatches) > maxMismatches:
                break
    return len(misMatches) <= maxMismatches

def getMatches(genome, fragment, maxMismatches):
    matches = []
    for i in range(len(genome) - len(fragment) + 1):
        misMatches = getMismatches(genome, fragment, offset)
        if isMatchAt(genome, fragment, i, maxMismatches):
            matches.append(i)
    return matches

def getAllMatches(genome, fragment, maxMismatches = 0):
    matches = getMatches(genome, fragment, maxMismatches)
    rc = reverseComplement(fragment)
    rcMatches = getMatches(genome, rc, maxMismatches) if rc != fragment else []
    return matches, rcMatches

In [ ]:
genome[36382:36382+4]

In [ ]:
map(len, getAllMatches(genome,'AGGT'))

In [ ]:
map(len, getAllMatches(genome,'TTAA'))

In [ ]:
reverseComplement('TTAA')

In [ ]:
print getAllMatches(genome, 'ACTAAGT')

In [ ]:
print getAllMatches(genome, 'AGTCGA', 0)

In [ ]:
print getAllMatches('ACTTACTTGATAAAGT', 'ACTTTA', 2)

In [ ]:
print len(getAllMatches(genome, 'TTCAAGCC', 2)[0])

In [ ]:
print getAllMatches(genome, 'AGGAGGTT', 2)[0]

In [ ]:
def charToQual(c):
    return ord(c) - 33

def loadFastq(filename):
    reads = []
    quals = []
    with open(filename) as f:
        while True:
            id_line = f.readline().rstrip()
            if not id_line:
                break
            assert id_line.startswith('@')
            reads.append(f.readline().rstrip())
            assert f.readline().rstrip() == '+'
            quals.append(map(charToQual, f.readline().rstrip()))
            assert len(reads[-1]) == len(quals[-1])
    return reads, quals

In [ ]:
r,q = loadFastq('ERR037900_1.first1000.fastq')

In [ ]:
len(r)

In [ ]:
print q[0]

In [ ]:
import numpy as n

In [ ]:
q = n.array(q)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(q.sum(0))

In [ ]:
plt.plot(q.std(0))

In [ ]:
q.sum(0).argmin()

In [ ]:
q.sum(0)[62:72]

There is a clearly lower quality value at index 66.

I'm going to do another analysis now, by seeing which indices in the reads most frequently cause a mismatch.

In [ ]:
x=set()

In [ ]:
x.update([1,2,3])

In [ ]:
x